#DAT405 Introduction to Data Science and AI 
##2022-2023, Reading Period 1
## Assignment 5: Reinforcement learning and classification
There will be an overall grade for this assignment. To get a pass grade (grade 5), you need to pass items 1-3 below. To receive higher grades, finish items 4 and 5 as well.

The exercise takes place in a notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical. 
Hints:
You can execute certain linux shell commands by prefixing the command with `!`. You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results the second you can use writing code snippets that execute the tasks required.  

This assignment is about **sequential decision making** under uncertainty (Reinforcement learning). In a sequential decision process, the process jumps between different states (the environment), and in each state the decision maker, or agent, chooses among a set of actions. Given the state and the chosen action, the process jumps to a new state. At each jump the decision maker receives a reward, and the objective is to find a sequence of decisions (or an optimal policy) that maximizes the accumulated rewards.

We will use **Markov decision processes** (MDPs) to model the environment, and below is a primer on the relevant background theory. The assignment can be divided in two parts:



* To make things concrete, we will first focus on decision making under **no** uncertainity, i.e, given we have a world model, we can calculate the exact and optimal actions to take in it. We will first introduce **Markov Decision Process (MDP)** as the world model. Then we give one algorithm (out of many) to solve it.


* Next, we will work through one type of reinforcement learning algorithm called Q-learning. Q-learning is an algorithm for making decisions under uncertainity, where uncertainity is over the possible world model (here MDP). It will find the optimal policy for the **unknown** MDP, assuming we do infinite exploration.

## Primer
### Decision Making
The problem of **decision making under uncertainty** (commonly known as **reinforcement learning**) can be broken down into
two parts. First, how do we learn about the world? This involves both the
problem of modeling our initial uncertainty about the world, and that of drawing conclusions from evidence and our initial belief. Secondly, given what we
currently know about the world, how should we decide what to do, taking into
account future events and observations that may change our conclusions?
Typically, this will involve creating long-term plans covering possible future
eventualities. That is, when planning under uncertainty, we also need to take
into account what possible future knowledge could be generated when implementing our plans. Intuitively, executing plans which involve trying out new
things should give more information, but it is hard to tell whether this information will be beneficial. The choice between doing something which is already
known to produce good results and experiment with something new is known
as the **exploration-exploitation dilemma**.

### The exploration-exploitation trade-off

Consider the problem of selecting a restaurant to go to during a vacation. Lets say the
best restaurant you have found so far was **Les Epinards**. The food there is
usually to your taste and satisfactory. However, a well-known recommendations
website suggests that **King’s Arm** is really good! It is tempting to try it out. But
there is a risk involved. It may turn out to be much worse than **Les Epinards**,
in which case you will regret going there. On the other hand, it could also be
much better. What should you do?
It all depends on how much information you have about either restaurant,
and how many more days you’ll stay in town. If this is your last day, then it’s
probably a better idea to go to **Les Epinards**, unless you are expecting **King’s
Arm** to be significantly better. However, if you are going to stay there longer,
trying out **King’s Arm** is a good bet. If you are lucky, you will be getting much
better food for the remaining time, while otherwise you will have missed only
one good meal out of many, making the potential risk quite small.

### Markov Decision Processes
Markov Decision Processes (MDPs) provide a mathematical framework for modeling sequential decision making under uncertainty. An *agent* moves between *states* in a *state space* choosing *actions* that affects the transition probabilities between states, and the subsequent *rewards* recieved after a jump. This is then repeated a finite or infinite number of epochs. The objective, or the *solution* of the MDP, is to optimize the accumulated rewards of the process.

Thus, an MDP consists of five parts: 

* Decision epochs: $t={1,2,...,T}$, where $T\leq \infty$
* State space: $S=\{s_1,s_2,...,s_N\}$ of the underlying environment
* Action space $A=\{a_1,a_2,...,a_K\}$ available to the decision maker at each decision epoch
* Transition probabilities $p(s_{t+1}|s_t,a_t)$ for jumping from state $s_t$ to state $s_{t+1}$ after taking action $a_t$
* Reward functions $R_t = r(a_t,s_t,s_{t+1})$ resulting from the chosen action and subsequent transition

A *decision policy* is a function $\pi: s \rightarrow a$, that gives instructions on what action to choose in each state. A policy can either be *deterministic*, meaning that the action is given for each state, or *randomized* meaning that there is a probability distribution over the set of possible actions for each state. Given a specific policy $\pi$ we can then compute the the *expected total reward* when starting in a given state $s_1 \in S$, which is also known as the *value* for that state, 

$$V^\pi (s_1) = E\left[ \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) {\Large |} s_1\right] = \sum_{t=1}^{T} r(s_t,a_t,s_{t+1}) p(s_{t+1} | a_t,s_t)$$ 

where $a_t = \pi(s_t)$. To ensure convergence and to control how much credit to give to future rewards, it is common to introduce a *discount factor* $\gamma \in [0,1]$. For instance, if we think all future rewards should count equally, we would use $\gamma = 1$, while if we value near-future rewards higher than more distant rewards, we would use $\gamma < 1$. The expected total *discounted* reward then becomes

$$V^\pi( s_1) = \sum_{t=1}^T \gamma^{t-1} r(s_t,a_t, s_{t+1}) p(s_{t+1} | s_t, a_t) $$

Now, to find the *optimal* policy we want to find the policy $\pi^*$ that gives the highest total reward $V^*(s)$ for all $s\in S$. That is, we want to find the policy where

$$V^*(s) \geq V^\pi(s), s\in S$$

To solve this we use a dynamic programming equation called the *Bellman equation*, given by

$$V(s) = \max_{a\in A} \left\{\sum_{s'\in S} p(s'|s,a)( r(s,a,s') +\gamma V(s')) \right\}$$

It can be shown that if $\pi$ is a policy such that $V^\pi$ fulfills the Bellman equation, then $\pi$ is an optimal policy.

A real world example would be an inventory control system. The states could be the amount of items we have in stock, and the actions would be the amount of items to order at the end of each month. The discrete time would be each month and the reward would be the profit. 


## Question 1

The first question covers a deterministic MPD, where the action is directly given by the state, described as follows:

* The agent starts in state **S** (see table below)
* The actions possible are **N** (north), **S** (south), **E** (east), and **W** west. 
* The transition probabilities in each box are uniform. Note, however, that you cannot move outside the grid, thus all actions are not available in every box.
* When reaching **F**, the game ends (absorbing state).
* The numbers in the boxes represent the rewards you receive when moving into that box. 
* Assume no discount in this model: $\gamma = 1$
    
| | | |
|----------|----------|---------|
|-1 |1|**F**|
|0|-1|1|  
|-1 |0|-1|  
|**S**|-1|1|

Let $(x,y)$ denote the position in the grid, such that $S=(0,0)$ and $F=(2,3)$.

**1a)** What is the optimal path of the MDP above? Is it unique? Submit the path as a single string of directions. E.g. NESW will make a circle.

**1b)** What is the optimal policy (i.e. the optimal action in each state)?

**1c)** What is expected total reward for the policy in 1b)?


1a) EENNN, it is not unique. 

1b)
[[EAST, EAST, END]
 [EAST, EAST, NORTH]
 [EAST, EAST, NORTH]
 [EAST, EAST, NORTH]]
 
| | | |
|----------|----------|---------|
|E |E|**F**|
|E|E|N|  
|E|E|N|  
|E|E|N|

1c) 0

## Value Iteration

For larger problems we need to utilize algorithms to determine the optimal policy $\pi^*$. *Value iteration* is one such algorithm that iteratively computes the value for each state. Recall that for a policy to be optimal, it must satisfy the Bellman equation above, meaning that plugging in a given candidate $V^*$ in the right-hand side (RHS) of the Bellman equation should result in the same $V^*$ on the left-hand side (LHS). This property will form the basis of our algorithm. Essentially, it can be shown that repeated application of the RHS to any intial value function $V^0(s)$ will eventually lead to the value $V$ which statifies the Bellman equation. Hence repeated application of the Bellman equation will also lead to the optimal value function. We can then extract the optimal policy by simply noting what actions that satisfy the equation.    

The process of repeated application of the Bellman equation is what we here call the _value iteration_ algorithm. It practically procedes as follows:

```
epsilon is a small value, threshold
for x from i to infinity 
do
    for each state s
    do
        V_k[s] = max_a Σ_s' p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
    end
    if  |V_k[s]-V_k-1[s]| < epsilon for all s
        for each state s,
        do
            π(s)=argmax_a ∑_s′ p(s′|s,a)*(r(a,s,s′) + γ*V_k−1[s′])
            return π, V_k 
        end
end

```

**Example:** We will illustrate the value iteration algorithm by going through two iterations. Below is a 3x3 grid with the rewards given in each state. Assume now that given a certain state $s$ and action $a$, there is a probability 0.8 that that action will be performed and a probabilit 0.2 that no action is taken. For instance, if we take action **E** in state $(x,y)$ we will go to $(x+1,y)$ 80 percent of the time (given that that action is available in that state), and remain still 20 percent of the time. We will use have a discount factor $\gamma = 0.9$. Let the initial value be $V^0(s)=0$ for all states $s\in S$. 

| | | |  
|----------|----------|---------|  
|0|0|0|
|0|10|0|  
|0|0|0|  


**Iteration 1**: The first iteration is trivial, $V^1(s)$ becomes the $\max_a \sum_{s'} p(s'|s,a) r(s,a,s')$ since $V^0$ was zero for all $s'$. The updated values for each state become

| | | |  
|----------|----------|---------|  
|0|8|0|
|8|2|8|  
|0|8|0|  
  
**Iteration 2**:  
  
Staring with cell (0,0) (lower left corner): We find the expected value of each move:  
Action **S**: 0  
Action **E**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **N**: 0.8( 0 + 0.9 \* 8) + 0.2(0 + 0.9 \* 0) = 5.76  
Action **W**: 0

Hence any action between **E** and **N** would be best at this stage.

Similarly for cell (1,0):

Action **N**: 0.8( 10 + 0.9 \* 2) + 0.2(0 + 0.9 \* 8) = 10.88 (Action **N** is the maximizing action)  

Similar calculations for remaining cells give us:

| | | |  
|----------|----------|---------|  
|5.76|10.88|5.76|
|10.88|8.12|10.88|  
|5.76|10.88|5.76|  


## Question 2

**2a)** Code the value iteration algorithm just described here, and show the converging optimal value function and the optimal policy for the above 3x3 grid.

**2b)** Explain why the result of 2a) does not depend on the initial value $V_0$.

In [5]:
## assumes a 2d array
# input_array = [[0,8,0],
#                [8,2,8],
#                [0,8,0]]
input_array = [[0,0,0],
                [0,0,0],
                [0,0,0]]

reward_array = [[0,0,0],
                [0,10,0],
                [0,0,0]]
eps = 1

In [6]:
def single_state_step(state_array, reward_array, row_idx, col_idx, decay, p):
#     print(len(state_array))
    NORTH = 0 if row_idx + 1 >= 3 else p * (reward_array[row_idx+1][col_idx] + state_array[row_idx+1][col_idx] * decay) + (1-p) * (reward_array[row_idx][col_idx] + state_array[row_idx][col_idx] * decay) 
    SOUTH = 0 if row_idx - 1 < 0 else p * (reward_array[row_idx-1][col_idx] + state_array[row_idx-1][col_idx] * decay) + (1-p) * (reward_array[row_idx][col_idx] + state_array[row_idx][col_idx] * decay) 
    EAST = 0 if col_idx + 1 >= 3 else p * (reward_array[row_idx][col_idx+1] + state_array[row_idx][col_idx+1] * decay) + (1-p) * (reward_array[row_idx][col_idx] + state_array[row_idx][col_idx] * decay) 
    WEST = 0 if col_idx-1 < 0 else p * (reward_array[row_idx][col_idx-1] + state_array[row_idx][col_idx-1] * decay) + (1-p) * (reward_array[row_idx][col_idx] + state_array[row_idx][col_idx] * decay) 
    result = ["NORTH", NORTH]
    if SOUTH > result[1]:
        result = ["SOUTH", SOUTH]
    if EAST > result[1]:
        result = ["EAST", EAST]
    if WEST > result[1]:
        result = ["WEST", WEST]
    return result

In [7]:
# Assuming a 2d array
while True:
    all_less_than_eps = True
    res = []
    for i in range(len(input_array)):
        res_row = []
        for j in range(len(input_array[i])):
            max_new_state = single_state_step(input_array, reward_array, i, j, 0.9, 0.8)
            res_row.append(max_new_state[1])
            if abs(max_new_state[1] - input_array[i][j]) >= eps:
                all_less_than_eps = False
        res.append(res_row)

    input_array = res
    
    if all_less_than_eps:
        res_policy = []
        for i in range(len(input_array)):
            res_policy_row = []
            for j in range(len(input_array[i])):
                max_new_state = single_state_step(input_array, reward_array, i, j, 0.9, 0.8)
                res_policy_row.append(max_new_state[0])
            res_policy.append(res_policy_row)
        break

res, res_policy

([[37.27427782051261, 43.60951608957746, 37.27427782051261],
  [43.60951608957746, 39.71330221075598, 43.60951608957746],
  [37.27427782051261, 43.60951608957746, 37.27427782051261]],
 [['NORTH', 'NORTH', 'NORTH'],
  ['EAST', 'NORTH', 'WEST'],
  ['SOUTH', 'SOUTH', 'SOUTH']])

## Reinforcement Learning (RL)
Until now, we understood that knowing the MDP, specifically $p(s'|a,s)$ and $r(s,a,s')$ allows us to efficiently find the optimal policy using the value iteration algorithm. Reinforcement learning (RL) or decision making under uncertainity, however, arises from the question of making optimal decisions without knowing the true world model (the MDP in this case).

So far we have defined the value function for a policy through $V^\pi$. Let's now define the *action-value function*

$$Q^\pi(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^\pi(s')]$$

The value function and the action-value function are directly related through

$$V^\pi (s) = \max_a Q^\pi (s,a)$$

i.e, the value of taking action $a$ in state $s$ and then following the policy $\pi$ onwards. Similarly to the value function, the optimal $Q$-value equation is:

$$Q^*(s,a) = \sum_{s'} p(s'|a,s) [r(s,a,s') + \gamma V^*(s')]$$

and the relationship between $Q^*(s,a)$ and $V^*(s)$ is simply

$$V^*(s) = \max_{a\in A} Q^*(s,a).$$

## Q-learning

Q-learning is a RL-method where the agent learns about its unknown environment (i.e. the MDP is unknown) through exploration. In each time step *t* the agent chooses an action *a* based on the current state *s*, observes the reward *r* and the next state *s'*, and repeats the process in the new state. Q-learning is then a method that allows the agent to act optimally. Here we will focus on the simplest form of Q-learning algorithms, which can be applied when all states are known to the agent, and the state and action spaces are reasonably small. This simple algorithm uses a table of Q-values for each $(s,a)$ pair, which is then updated in each time step using the update rule in step $k+1$

$$Q_{k+1}(s,a) = Q_k(s,a) + \alpha \left( r(s,a) + \gamma \max \{Q_k(s',a')\} - Q_k(s,a) \right) $$ 

where $\gamma$ is the discount factor as before, and $\alpha$ is a pre-set learning rate. It can be shown that this algorithm converges to the optimal policy of the underlying MDP for certain values of $\alpha$ as long as there  is sufficient exploration. For our case, we set a constant $\alpha=0.1$.

## OpenAI Gym

We shall use already available simulators for different environments (worlds) using the popular [OpenAI Gym library](https://www.gymlibrary.dev/). It just implements different types of simulators including ATARI games. Although here we will only focus on simple ones, such as the **Chain enviroment** illustrated below.
![alt text](https://chalmersuniversity.box.com/shared/static/6tthbzhpofq9gzlowhr3w8if0xvyxb2b.jpg)
The figure corresponds to an MDP with 5 states $S = \{1,2,3,4,5\}$ and two possible actions $A=\{a,b\}$ in each state. The arrows indicate the resulting transitions for each state-action pair, and the numbers correspond to the rewards for each transition.

## Question 3
You are to first familiarize with the framework of [the OpenAI environments](https://www.gymlibrary.dev/), and then implement the Q-learning algorithm for the <code>NChain-v0</code> enviroment depicted above, using default parameters and a learning rate of $\gamma=0.95$. Report the final $Q^*$ table after convergence of the algorithm. For an example on how to do this, you can refer to the Q-learning of the **Frozen lake environment** (<code>q_learning_frozen_lake.ipynb</code>), uploaded on Canvas. Hint: start with a small learning rate.

Note that the NChain environment is not available among the standard environments, you need to load the <code>gym_toytext</code> package, in addition to the standard gym:

<code>
!pip install gym-legacy-toytext<br>
import gym<br>
import gym_toytext<br>
env = gym.make("NChain-v0")<br>
</code>

In [11]:
# %pip install gym-legacy-toytext 
# %pip install gym 

import gym
import gym_toytext

env = gym.make("NChain-v0")


In [12]:
# env = gym.make("NChain-v0", apply_api_compatibility=True)

In [13]:
# env = gym.wrappers.StepAPICompatibility(gym.make("NChain-v0"), output_truncation_bool=True)

In [14]:
env

<TimeLimit<OrderEnforcing<PassiveEnvChecker<NChainEnv<NChain-v0>>>>>

In [15]:
# saved = []
# for env in gym.envs.registry.env_specs:
# #      if 'KellyCoinflip' in env or 'NChain' in env:
# #       print('Remove {} from registry'.format(env))
#       saved.append(env)
# for name in saved:
#     del gym.envs.registry.env_specs[name]

In [4]:
env.observation_space

NameError: name 'env' is not defined

In [17]:
import numpy as np
import random

In [18]:
num_episodes = 15000 #20000 #60000
gamma = 0.95 #0.99
learning_rate = 0.1 #0.95 #0.85
epsilon = 0.5 #1 #0.15 #0.1

# initialize the Q table
Q = np.zeros([5, 2])

In [21]:
for i in range(num_episodes):
    state = env.reset()
#     env.render()
    done = False
    while done == False:
        # First we select an action:
        if random.uniform(0, 1) < epsilon: # Flip a skewed coin
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(Q[state,:]) # Exploit learned values
        #Then we perform the action and receive the feedback from the environment
        new_state, reward, done, info = env.step(action)
        # Finally we learn from the experience by updating the Q-value of the selected action
        update = reward + (gamma*np.max(Q[new_state,:])) - Q[state, action]
        Q[state,action] += learning_rate*update 
        state = new_state
    print(f"Q at episode {i}: {Q}")
    print()

KeyboardInterrupt: 

In [22]:
Q

array([[60.62708863, 60.27121276],
       [64.06344293, 62.3080182 ],
       [68.32830855, 63.41738084],
       [75.04869397, 68.4919782 ],
       [77.41411079, 66.09418357]])

## Question 4

**4a)** What is the importance of exploration in RL? Explain with an example.

**4b)** Explain what makes reinforcement learning different from supervised learning tasks such as regression or classification.


4a) Exploration is important as it allows us to find optimal solutions in the long term. If we only exploit, we are only always choosing the best options historically for short term reward and not trying new moves , that could lead to more optimal states and actions in the future. So to find optimal solutions, we need to explore while also exploit to get the best of both worlds. 

For example, an agent who is looking to get richer. He has the option of continuing to work at his fixed salary job and get a fixed income, or he could explore into other options like starting his own business, or moving to a new industry, and these options might not end well, i.e he might fail and lose his income, but there is a chance he might be very successful and maybe end up earning alot more money then his current job.

4b) Reinforcement learning is different from supervised learning tasks such as regression or classification whereby reinforcement learning does not require labeled dataset like the rest. For example, for classification tasks, both the input and outputs need to be fed to the model as training dataset so the model can learn, however reinforcement learning does not require corresponding outputs to the inputs and is essentially an agent that just interacts with an environment and tries to reach a desired outcome. If the agent ends in a desirable state, he is rewarded and if not, he is punished.

## Question 5

**5a)** Give a summary of how a decision tree works and how it extends to random forests.

**5b)** State at least one advantage and one drawback with using random forests over decision trees.

5a) Decision trees use a branching method to map an input to all possible outputs. It is trained on a set of training data to determine the branches. Let me illustrate this with an example,
task: given an input of {hasFeathers, canFly, hasFins} predict the animal
the tree will be constructed to look like:
- if the input hasFeathers == True, then check if can fly
    - if yes then it is a bird that can fly
    - if no then it is a bird that cant fly
- if the input hasFeathers == False, then check if it has Fins
    - if yes then it is a fish
    - if no then it is a mammal
    
    
It is essentially a set of if else statements to classify inputs.

The problem of decision tree is that it is very easy to overfit on the training set and thus affecting its ability to predict on new data accurately. Thus a solution to this is using random forests. Random forest essentially is made up of multiple decision trees. In our example above, we have input of 3 attributes, so we can permutate the attributes and create 3 different decision trees based on 2 of the attributes, i.e 1st tree input {hasFeathers, canFly}, 2nd tree input {hasFeathers, hasFins} and 3rd tree input {canFly, hasFins}. These three trees will be trained separately and combined together to form the random forest. When is it time to make a prediction, the input will be fed into all 3 trees individually and their prediction results will be consolidated and a count of each result will be tallied, and the majority vote will be the final output.

5b)
Advantage:
- it does not suffer from the overfitting problem since it takes the average of all the predictions, there is no bias

Disadvantage:
- it is slow in making predictions as it needs to run through many decision trees and then do a count and voting on it.


# References
Primer/text based on the following references:
* http://www.cse.chalmers.se/~chrdimi/downloads/book.pdf
* https://github.com/olethrosdc/ml-society-science/blob/master/notes.pdf